In [1]:
import pandas as pd
import numpy as np

/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/srv/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
pd.options.mode.chained_assignment = None  # default='warn'
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
db = pd.read_excel('Detections_of_IBC_2018_07_06.xlsx')

In [6]:
def define_db(route, border_type='Sea'):
    db_filtered = db[(db.Route == route) & (db.BorderLocation == border_type)]
    db_filtered.drop(['BorderLocation', 'Route'], axis=1, inplace=True)
    db_filtered.set_index('NationalityLong', inplace=True)
    
    cluster_now = db_filtered.iloc[:,-2:].sum(axis=1).sort_values(ascending=False)
    cluster_before = db_filtered.iloc[:,-4:-2].sum(axis=1).sort_values(ascending=False)
    
    top10_now = cluster_now[:10]
    top10_before = cluster_before[:10]
    
    return top10_now, top10_before

In [7]:
def calculate_results(top10_now, top10_before):
    nationalities = []
    nationalities_dropped = []
    top_num = 0
    
    for nat in top10_now.index:
        if nat in top10_before.index:
            nationality = [nat for nat in top10_before.index if nat == top10_now.index[top_num]]
            result = ((top10_now[nat]-top10_before[nat])/top10_before[nat])*100.0
            nationalities.append([nat, round(result,1), top10_before[nat], top10_now[nat]])
        else:
            nationalities.append([nat, str('NEW (' + str(top10_now[nat])) + ')'])
            top_num += 1
    
    for nat in top10_before.index:
        if nat not in top10_now.index:
            nationalities_dropped.append(nat)
    
    return nationalities, nationalities_dropped

In [9]:
def show_result(result, result_dropped):
    for item in result:
        if isinstance(item[1], float) and not np.isnan(item[1]):
            print(item[0] + ', '
                  + str(item[1]) + '% '
                  + '(' + str(item[2]) + ' -> '
                  + str(item[3]) + ')')
        else:
            print(item[0] + ', ' + str(item[1]))
    
    for item in result_dropped:
        print('{} dropped from top 10.'.format(item))

In [16]:
def main(route, border_type='Sea'):
    print('\nResults for {}.\n'.format(route))
    now, before = define_db(route, border_type)
    final, final_dropped = calculate_results(now, before)
    show_result(final, final_dropped)

In [12]:
routes = np.unique(db.Route.values)
routes

array(['Black Sea', 'Central Mediterranean',
       'Circular Route from Albania to Greece', 'Eastern Land Borders',
       'Eastern Mediterranean', 'Other', 'Western African',
       'Western Balkans', 'Western Mediterranean'], dtype=object)

In [17]:
print('\n--- FOR SEA ---\n')
for route in routes:
    main(route)
    
print('\n--- FOR LAND ---\n')
for route in routes:
    main(route, 'Land')


--- FOR SEA ---


Results for Black Sea.

Morocco, nan
Other and stateless, nan
Turkey, nan
Algeria, nan
Somalia, nan
Iran, nan
Belarus, nan
Afghanistan, nan
Syria, nan
Pakistan, nan

Results for Central Mediterranean.

Tunisia, 172.4% (587 -> 1599)
Sudan, 689.6% (106 -> 837)
Eritrea, 82.7% (370 -> 676)
Nigeria, 244.4% (189 -> 651)
Côte d'Ivoire, 896.7% (60 -> 598)
Mali, 422.0% (91 -> 475)
Guinea, 252.0% (98 -> 345)
Algeria, NEW (254)
Senegal, NEW (145)
Pakistan, NEW (145)
Somalia dropped from top 10.
Niger dropped from top 10.
Iraq dropped from top 10.

Results for Circular Route from Albania to Greece.


Results for Eastern Land Borders.


Results for Eastern Mediterranean.

Syria, 88.5% (1479 -> 2788)
Iraq, 25.5% (970 -> 1217)
Afghanistan, 54.7% (411 -> 636)
Congo (D.R.), 267.3% (52 -> 191)
Palestine, 100.0% (88 -> 176)
Cameroon, 102.4% (84 -> 170)
Pakistan, NEW (162)
Somalia, 252.8% (36 -> 127)
Iran, 52.3% (65 -> 99)
Eritrea, NEW (80)
Algeria dropped from top 10.
Yemen dropped fro